# Install packages and libraries

In [ ]:
!pip install GeneralisedFormanRicci
!pip install gudhi
!pip install plotly

In [ ]:
import functools
import pandas as pd
import numpy as np 
import gudhi as gd
import networkx as nx
import plotly.graph_objects as go
import math
import matplotlib as mpl
import matplotlib
import plotly.io as pio
import matplotlib.pyplot as plt
from GeneralisedFormanRicci.frc import gen_graph
from scipy.sparse import *
from scipy import *

# Define functions

In [ ]:
def faces(simplices):
    faceset = set()
    for simplex in simplices:
        numnodes = len(simplex)
        for r in range(numnodes, 0, -1):
            for face in combinations(simplex, r):
                faceset.add(tuple(sorted(face)))
    return faceset

def n_faces(face_set, n):
    return filter(lambda face: len(face)==n+1, face_set)

def boundary_operator(face_set, i):
    source_simplices = list(n_faces(face_set, i))
    target_simplices = list(n_faces(face_set, i-1))
    #print(source_simplices, target_simplices)

    if len(target_simplices)==0:
        S = dok_matrix((1, len(source_simplices)), dtype=np.float64)
        S[0, 0:len(source_simplices)] = 1
    else:
        source_simplices_dict = {source_simplices[j]: j for j in range(len(source_simplices))}
        target_simplices_dict = {target_simplices[i]: i for i in range(len(target_simplices))}

        S = dok_matrix((len(target_simplices), len(source_simplices)), dtype=np.float64)
        for source_simplex in source_simplices:
            for a in range(len(source_simplex)):
                target_simplex = source_simplex[:a]+source_simplex[(a+1):]
                i = target_simplices_dict[target_simplex]
                j = source_simplices_dict[source_simplex]
                S[i, j] = -1 if a % 2==1 else 1
    
    return S

In [ ]:
import os
xyz_files = []
for x in os.listdir():
    if x.endswith(".xyz"):
        xyz_files.append(x)

In [ ]:
np.shape(xyz_files)

In [ ]:
veca = []
for filename in xyz_files:
    vec = [filename]
    atoms = []
    coords = []

    xyz = open(filename)
    n_atoms = int(xyz.readline())
    title = xyz.readline()
    for line in xyz:
        atom,x,y,z = line.split()
        atoms.append(atom)
        coords.append([float(x), float(y), float(z)])
    xyz.close()
    
    
    for f in np.round(np.arange(0,7.5,0.5),1):
        rc = gd.RipsComplex(coords, max_edge_length=f)
        simplex_tree = rc.create_simplex_tree(max_dimension=2)
        val = simplex_tree.get_filtration()
        simplices = set()
        for v in val:
            simplices.add(tuple(v[0]))
        
        laplacian_0 = np.matmul(boundary_operator(simplices, 1).toarray(), np.transpose(boundary_operator(simplices, 1).toarray()))
        laplacian_1 = np.matmul(boundary_operator(simplices, 2).toarray(), np.transpose(boundary_operator(simplices, 2).toarray())) + np.matmul(np.transpose(boundary_operator(simplices, 1).toarray()), boundary_operator(simplices, 1).toarray())     
        #laplacian_2 = np.matmul(boundary_operator(simplices, 1).toarray(), np.transpose(boundary_operator(simplices, 1).toarray())) + np.matmul(np.transpose(boundary_operator(simplices, d-1).toarray()), boundary_operator(simplices, d-1).toarray())

        
        # Attributes for Laplacian Dim(0) for each filtration value
        eigval_0, eigvec_0 = np.linalg.eigh(laplacian_0)
        eigval_0 = [0 if i <1e-3 else i for i in eigval_0]
        b_0 = [j for j in eigval_0 if j==0]
        b_0_positive = [j for j in eigval_0 if j>0]
        b_0_positive_rp = np.reciprocal(b_0_positive)
        A_0 = len(b_0_positive)
        ab_std_0 = np.absolute(eigval_0-np.average(eigval_0))


        beta_0 = len(b_0) #Attribute1
        mean_0 = np.average(eigval_0) #Attribute2
        std_0 = np.std(eigval_0) #Attribute3
        sum_0 = np.sum(eigval_0) #Attribute4
        min_0 = np.amin(eigval_0) #Attribute5
        max_0 = np.amax(eigval_0) #Attribute6
        ab_std_sum_0 = np.sum(ab_std_0) #Attribute7
        order_2_sum_0 = np.sum(np.square(eigval_0)) #Attribute8
        qwi_0 = np.sum(np.multiply(b_0_positive_rp,(A_0+1))) #Attribute9
        #stn_0 = np.log(np.multiply(np.prod(b_0_positive),1/(A_0+1))) #Attribute10 This gives inf values
        stn_0 = np.sum(np.log(b_0_positive)) - np.log(A_0+1) #Attribute10

        
        # Attributes for Laplacian Dim(1) for each filtration value
        eigval_1, eigvec_1 = np.linalg.eigh(laplacian_1)
        eigval_1 = [0 if i <1e-3 else i for i in eigval_1]
        b_1 = [j for j in eigval_1 if j==0]
        b_1_positive = [j for j in eigval_1 if j>0]
        b_1_positive_rp = np.reciprocal(b_1_positive)
        A_1 = len(b_1_positive)
        ab_std_1 = np.absolute(eigval_1-np.average(eigval_1))

        beta_1 = len(b_1) #Attribute1
        mean_1 = 0 if np.shape(laplacian_1)==(0,0) else np.average(eigval_1) #Attribute2
        std_1 = 0 if np.shape(laplacian_1)==(0,0) else np.std(eigval_1) #Attribute3
        sum_1 = 0 if np.shape(laplacian_1)==(0,0) else np.sum(eigval_1) #Attribute4
        min_1 = 0 if np.shape(laplacian_1)==(0,0) else np.amin(eigval_1) #Attribute5
        max_1 = 0 if np.shape(laplacian_1)==(0,0) else np.amax(eigval_1) #Attribute6
        ab_std_sum_1 = np.sum(ab_std_1) #Attribute7
        order_2_sum_1 = np.sum(np.square(eigval_1)) #Attribute8
        qwi_1 = np.sum(np.multiply(b_1_positive_rp,(A_1+1))) #Attribute9
        #stn_1 = np.log(np.multiply(np.prod(b_1_positive),1/(A_1+1))) #Attribute10 This gives inf values
        stn_1 = np.sum(np.log(b_1_positive)) - np.log(A_1+1) #Attribute10
        
        
        vec.extend([beta_0,beta_1,std_0,std_1,sum_0,sum_1,min_0,min_1,max_0,max_1,mean_0,mean_1,ab_std_sum_0,ab_std_sum_1,order_2_sum_0,order_2_sum_1,qwi_0,qwi_1,stn_0,stn_1])
             
    veca.append(vec)    
    

In [ ]:
df = pd.DataFrame(data=veca)
df.to_csv('hodge_fingerprint_new_spn_edited.csv', index=False, header=False)
df

In [ ]:
filename

In [ ]:
ab_std_0 = np.absolute(eigval_0-np.average(eigval_0))
ab_std_sum_0 = np.sum(ab_std_0)
ab_std_sum_0

In [ ]:
np.average(eigval_1)